# VencoPy Tutorial 1

This tutorial showcases the general structure and workflow of venco.py, as well as some basic features of its 6 main classes:
- DataParser
- GridModeller
- FlexEstimator
- DiaryBuilder
- Aggregator
- PostProcessor

All tutorials run on a very small subset of data from the 2017 German national travel survey (Mobilität in Deutschland (MiD17)), which might result in profiles having uncommon shapes. As such, the calculations and the examples proposed throughout all tutorials have the mere goal to exemplify the modelling steps and guide the use throughout the structure of venco.py and do not aim at providing an accurate quantification of demand-side flexibility from EVs.

For a more detailed description of venco.py, you can refer to the documentation at https://dlr-ve.gitlab.io/esy/vencopy/vencopy/

## Setting up the working space

This section allows you to import all required Python packages for data input and manipulation. The function os.chdir(path) allows us to point Python towards the top most directory which contains all useful venco.py functions that are going to be used in the tutorials.
Additionally, we set and read in the input dataframe (here the MiD17) and load the necessary yaml file, which contains some configuration settings.

In [43]:
import time
from pathlib import Path

In [44]:
from vencopy.core.dataparsers import parse_data
from vencopy.core.gridmodellers import GridModeller
from vencopy.core.flexestimators import FlexEstimator
from vencopy.core.diarybuilders import DiaryBuilder
from vencopy.core.profileaggregators import ProfileAggregator
from vencopy.core.postprocessors import PostProcessor
from vencopy.utils.utils import load_configs, create_output_folders

start_time = time.time()

We will have a look more in detail at each config file and what you can specify within it for each class throughtout the tutorials. For the time being, it is enough to know that the config files specify configurations, variable namings and settings for the different classes. There is one config file for each class, a global config and a local configuration config to specify eventual file paths on your machine.


In [45]:
base_path = Path.cwd().parent
configs = load_configs(base_path)

## _DataParser_ class

To be able to estimate EV electric consumption and flexibililty, the first step in the venco.py framework implies accessing a travel survey data set, such as the MiD. This is carried out through a parsing interface to the original database. In the parsing interface to the data set, three main operations are carried out: the read-in of the travel survey trip data, stored in .dta or .csv files, filtering and cleaning of the original raw data set and a set of variable replacement operations to allow the composition of travel diaries in a following step (in the DiaryBuilder class).


In order to have consistent entry data for all variables and for different data sets, all database entries are harmonised, which includes generating unified data types and consistent variable naming. The naming convention for the variables and their respective input type can be specified in the venco.py config files that have been loaded previously.

First off, we modify the localConfig and globalConfig files so that it point to the current working directory and to the database subset we will use to explain the different classes.


In [46]:
# Adapt relative paths in config for tutorials
configs['dev_config']['global']['relative_path']['parse_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['parse_output']
configs['dev_config']['global']['relative_path']['diary_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['diary_output']
configs['dev_config']['global']['relative_path']['grid_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['grid_output']
configs['dev_config']['global']['relative_path']['flex_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['flex_output']
configs['dev_config']['global']['relative_path']['aggregator_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['aggregator_output']
configs['dev_config']['global']['relative_path']['processor_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['processor_output']


# Set reference dataset
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configs['user_config']['global']['absolute_path'][datasetID] = Path.cwd() /'data_sampling'

# Similarly we modify the datasetID in the global config file
configs['dev_config']['global']['files'][datasetID]['trips_data_raw'] = datasetID + '.csv'

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configs['dev_config']['dataparsers']['data_variables']['household_id']
del configs['dev_config']['dataparsers']['data_variables']['person_id']

In [47]:
create_output_folders(configs=configs)

We can now run the first class and parse the dataset with the collection of mobility patterns into a more useful form for our scope.

In [48]:
data = parse_data(configs=configs)
data.process()

Generic file parsing properties set up.
Starting to retrieve local data file from c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\tutorials\data_sampling\MiD17.csv.
Finished loading 2124 rows of raw data of type .csv.
Running in debug mode.
Finished harmonization of variables.
Finished harmonization of ID variables.
Starting filtering, applying 8 filters.
All filters combined yielded that a total of 241 trips are taken into account.
This corresponds to 40.166666666666664 percent of the original data.
Completed park timestamp adjustments.
Finished activity composition with 240 trips and 240 parking activites.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\dataparser\vencopy_output_dataparser_None_MiD17.csv.
Parsing MiD dataset completed.


,index,unique_id,park_id,trip_id,is_driver,household_person_id,trip_weight,trip_scale_factor,trip_purpose,trip_distance,...,timestamp_start,timestamp_end,is_first_activity,is_last_activity,activity_id,next_activity_id,previous_activity_id,is_first_trip,is_first_park_activity,time_delta
0,0,1410,0.0,NaN,True,1410,0.146627,39.299224,2,NaN,...,2017-03-13 00:00:00,2017-03-13 14:00:00,True,False,0.0,4.0,NaN,False,True,0 days 14:00:00
1,1,1410,NaN,4.0,True,1410,0.146627,39.299224,2,9.50,...,2017-03-13 14:00:00,2017-03-13 14:50:00,False,False,4.0,4.0,0.0,True,False,0 days 00:50:00
2,2,1410,4.0,NaN,True,1410,0.146627,39.299224,2,NaN,...,2017-03-13 14:50:00,2017-03-14 00:00:00,False,True,4.0,NaN,4.0,False,False,0 days 09:10:00
3,3,636,0.0,NaN,True,636,0.190006,50.925648,1,NaN,...,2016-07-24 00:00:00,2016-07-24 06:10:00,True,False,0.0,1.0,NaN,False,True,0 days 06:10:00
4,4,636,NaN,1.0,True,636,0.190006,50.925648,1,13.30,...,2016-07-24 06:10:00,2016-07-24 06:30:00,False,False,1.0,1.0,0.0,True,False,0 days 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,556,1840,0.0,NaN,True,1840,2.340718,627.363646,7,NaN,...,2017-03-27 00:00:00,2017-03-27 09:45:00,True,False,0.0,1.0,NaN,False,True,0 days 09:45:00
556,557,1840,NaN,1.0,True,1840,2.340718,627.363646,7,4.75,...,2017-03-27 09:45:00,2017-03-27 10:05:00,False,False,1.0,1.0,0.0,True,False,0 days 00:20:00
557,558,1840,1.0,NaN,True,1840,2.340718,627.363646,7,NaN,...,2017-03-27 10:05:00,2017-03-27 11:30:00,False,False,1.0,2.0,1.0,False,False,0 days 01:25:00
558,559,1840,NaN,2.0,True,1840,2.340718,627.363646,8,4.75,...,2017-03-27 11:30:00,2017-03-27 11:40:00,False,False,2.0,2.0,1.0,False,False,0 days 00:10:00


## _GridModeller_ class

The charging infrastructure allocation makes use of a basic charging infrastructure model, which assumes the availability of charging stations when vehicles are parked. Since the analytical focus of the framework lies on a regional level (NUTS1-NUTS0), the infrastructure model is kept simple in the current version.


Charging availability is allocated based on a binary True–False mapping to a respective trip purpose in the venco.py config. Thus, different scenarios describing different charging availability scenarios, e.g., at home or at home and at work etc. can be distinguished, but neither a regional differentiation nor a charging availability probability or distribution are assumed.

At the end of the execution of the GridModeller class, a column representing the available charging power is added to the activities dataset.

In [49]:
grid = GridModeller(configs=configs, activities=data.activities)
grid.assign_grid()

Starting with charge connection replacement of location purposes.
Grid connection assignment complete.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\gridmodeller\vencopy_output_gridmodeller_None_MiD17.csv.


## _FlexEstimator_ class

The flexEstimator class is the final class that is used to estimate the charging flexibility based on driving profiles and charge connection shares.
There are three integral inputs to the flexibililty estimation:
- A profile describing driven distances for each vehicle
- A profiles describing the available charging power if a vehicle is connected to the grid
- Techno–economic input assumptions

In [50]:
flex = FlexEstimator(configs=configs, activities=grid.activities)
flex.estimate_technical_flexibility_through_iteration()

Starting maximum battery level calculation.
Calculating maximum battery level for first activities.
Calculating maximum battery level for act 1.
Calculating maximum battery level for act 2.
Calculating maximum battery level for act 3.
Calculating maximum battery level for act 4.
Calculating maximum battery level for act 5.
Calculating maximum battery level for act 6.
Calculating maximum battery level for act 7.
Calculating maximum battery level for act 8.
Calculating maximum battery level for act 9.
Calculating maximum battery level for act 10.
Calculating maximum battery level for act 11.
Starting minimum battery level calculation.
Calculate minimum battery level for last activities.
Calculate minimum battery level for act 11.
Calculate minimum battery level for act 10.
Calculate minimum battery level for act 9.
Calculate minimum battery level for act 8.
Calculate minimum battery level for act 7.
Calculate minimum battery level for act 6.
Calculate minimum battery level for act 5.
Cal

,unique_id,index,park_id,trip_id,is_driver,household_person_id,trip_weight,trip_scale_factor,trip_purpose,trip_distance,...,min_battery_level_end_unlimited,max_residual_need,min_residual_need,max_overshoot,min_undershoot,auxiliary_fuel_need,drain,max_charge_volume,min_battery_level_start_unlimited,residual_need
0,56,97,0.0,NaN,True,56,0.233159,62.491608,8,NaN,...,None,None,None,158.4,0,NaN,NaN,158.400,-155.19,NaN
1,56,98,NaN,3.0,True,56,0.233159,62.491608,8,9.50,...,None,0,0,None,None,NaN,1.7100,NaN,3.21,NaN
2,56,99,3.0,NaN,True,56,0.233159,62.491608,8,NaN,...,None,None,None,72.54,None,NaN,NaN,74.250,NaN,NaN
3,75,282,0.0,NaN,True,75,1.418687,380.239194,4,NaN,...,None,None,None,158.4,0,NaN,NaN,158.400,-156.5922,NaN
4,75,283,NaN,3.0,True,75,1.418687,380.239194,4,1.71,...,None,0,0,None,None,NaN,0.3078,NaN,1.8078,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,1996,135,0.0,NaN,True,1996,0.416754,111.699169,7,NaN,...,None,None,None,158.4,0,NaN,NaN,158.400,-156.4896,NaN
457,1996,136,NaN,3.0,True,1996,0.416754,111.699169,7,2.28,...,None,0,0,None,None,NaN,0.4104,NaN,1.9104,NaN
458,1996,137,3.0,NaN,True,1996,0.416754,111.699169,7,NaN,...,None,None,None,8.6646,0,NaN,NaN,9.075,-7.1646,NaN
459,1996,138,NaN,4.0,True,1996,0.416754,111.699169,8,2.28,...,None,0,0,None,None,NaN,0.4104,NaN,1.9104,NaN


## _DiaryBuilder_ class

In the DiaryBuilder, individual trips at the survey day are consolidated into person-specific travel diaries comprising multiple trips.


The daily travel diary composition consists of three main steps: reformatting the database, allocating trip purposes and merging the obtained dataframe with other relevant variables from the original database.


In the first step, reformatting, the time dimension is transferred from the raw data (usually in minutes) to the necessary output format (e.g., hours). Each trip is split into shares, which are then assigned to the respective hour in which they took place, generating an hourly dataframe with a timestamp instead of a dataframe containing single trip entries.


Similarly, miles driven and the trip purpose are allocated to their respective hour and merged into daily travel diaries. Trips are assumed to determine the respective person’s stay in the consecutive hours up to the next trip and therefore are related to the charging availability between two trips. Trip purposes included in surveys may comprise trips carried out for work or education reasons, trips returning to home, trips to shopping facilities and other leisure activities. Currently, trips whose purpose is not specified are allocated to trips returning to their own household.

In [51]:
diary = DiaryBuilder(configs=configs, activities=flex.activities)
diary.create_diaries()

43 activities dropped from 461 total activities because activity length equals zero.
Starting to discretise drain.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\diarybuilder\vencopy_output_diarybuilder_None_drain_MiD17.csv.
Discretisation finished for drain.
Needed time to discretise drain: 0.21003293991088867.
Starting to discretise available_power.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\diarybuilder\vencopy_output_diarybuilder_None_available_power_MiD17.csv.
Discretisation finished for available_power.
Needed time to discretise available_power: 0.19718575477600098.
Starting to discretise uncontrolled_charging.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\diarybuilder\vencopy_output_diarybuilder_None_uncontrolled_charging_MiD17.csv.
Discretisation finished for uncontrolled_charging.
Needed time to discretise uncont

## _ProfileAggregator_ class

In [52]:

profiles = ProfileAggregator(configs=configs, activities=diary.activities, profiles=diary)
profiles.aggregate_profiles()

Starting to aggregate drain to fleet level based on day of the week.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\profileaggregator\vencopy_output_profileaggregator_None_drain_MiD17.csv.
Aggregation finished for drain.
Needed time to aggregate drain: 0.027998924255371094.
Starting to aggregate charging_power to fleet level based on day of the week.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\profileaggregator\vencopy_output_profileaggregator_None_charging_power_MiD17.csv.
Aggregation finished for charging_power.
Needed time to aggregate charging_power: 0.026000499725341797.
Starting to aggregate uncontrolled_charging to fleet level based on day of the week.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\profileaggregator\vencopy_output_profileaggregator_None_uncontrolled_charging_MiD17.csv.
Aggregation finished for uncont

## _PostProcessor_ class

In [53]:
post = PostProcessor(configs=configs, profiles=profiles)
post.create_annual_profiles()
post.normalise()

Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_drain_MiD17.csv.
Run finished.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_uncontrolled_charging_MiD17.csv.
Run finished.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_charging_power_MiD17.csv.
Run finished.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_max_battery_level_MiD17.csv.
Run finished.
Dataset written to c:\Users\mior_fa\Documents\7_Work\vencopy\vencopy_internal\vencopy\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_min_battery_level_MiD17.csv.
Run finished.
Dataset written 

In [54]:
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}.")

Elapsed time: 6.929261207580566.


## Next Steps

In the next tutorials, you will learn more in detail the internal workings of each class and how to customise some settings.